======================== Import Packages ==========================

In [9]:
import sys, os, pdb
import numpy as np
import pandas as pd 
from astropy.table import Table, vstack, Column, join
import calc_dust_masses
import csv
from lifelines import KaplanMeierFitter
from IPython.display import display

========================== Define Fuctions ==========================

In [10]:
def get_usc(f1, f2):

    """
    PURPOSE:    Create table for Upper Sco with info needed to run the 
                two-sample tests and the KME for comparing regions

                Calculates dust masses in same way as in Ansdell+2016
                Replaces non-detections with 3-sigma upper limits
    INPUT:      f1 = path to table 1 from Barenfeld+2016 (2016ApJ...827..142B)
                f2 = path to table 4 from Barenfeld+2016 (2016ApJ...827..142B)

    OUTPUT:     t = table that contains:
                  Source names (str)
                  Detection flags (int; 1=detection, 0=non-detection)
                  Dust masses with non-detections set to 3-sigma upper limits
                  Continuum fluxes with non-detections set to 3-sigma upper limits
                  Stellar masses and spectral types

    """

    ### GRAB UPPER SCO DATA
    t1 = Table.read(f1, format='ascii.cds')
    t2 = Table.read(f2, format='ascii.cds')
    t = join(t1, t2, join_type='inner')
    t['Source'].name = 'Name'

    ### FLAG (NON-)DETECTIONS
    ### REPLACE NON-DETECTION FLUXES WITH 3-SIGMA UPPER LIMITS
    t['det'] = np.repeat(0, len(t))
    t['det'][t['Snu'] / t['e_Snu'] >= 3.0] = 1
    t['Snu'][np.where(t['det'] == 0)] = 3.0 * t['e_Snu'][np.where(t['det'] == 0)]
    
    ### ONLY KEEP MSTAR >= 0.1
    t = t[np.where(10**t['logM'] >= 0.1)]

    ### CALCULATE DUST MASSES USING SAME METHOD AS LUPUS
    mdust = []
    for i, val in enumerate(t):
        mdust.append(calc_dust_masses.get_dustmass(1.0, 880., t['Snu'][i], 145., 20.))
    t['MDust'] = mdust

    ### ONLY KEEP "PRIMORDIAL" DISKS 
    ### TO MATCH SAMPLES OF LUPUS & TAURUS
    t = t[np.where( (t['Type'] == 'Full') | (t['Type'] == 'Transitional') | (t['Type'] == 'Evolved'))] 

    ### CLEAN UP
    t['Snu'].name = 'FCont'
    t['Mstar'] = 10**t['logM']

    return t['Name', 'SpT', 'Mstar', 'FCont', 'MDust', 'det']

In [11]:
def get_lup(f1, f2):

    """
    PURPOSE:    Create table for Lupus with info needed to run the 
                two-sample tests and the KME for comparing regions

                Calculates dust masses in same way as in Ansdell+2016
                Replaces non-detections with 3-sigma upper limits

    INPUT:      f1 = path to table 1 from Ansdell+2016 (2016ApJ...828...46A)
                f2 = path to table 2 from Ansdell+2016 (2016ApJ...828...46A)

    OUTPUT:     t = table that contains:
                  Source names (str)
                  Detection flags (int; 1=detection, 0=non-detection)
                  Dust masses with non-detections set to 3-sigma upper limits
                  Continuum fluxes with non-detections set to 3-sigma upper limits
                  Stellar masses and spectral types
                  Distances
    """

    t1 = Table.read(f1, format='ascii.cds')
    t2 = Table.read(f2, format='ascii.cds')
    t = join(t1, t2, join_type='inner')

    ### ONLY KEEP KNOWN STELLAR MASSES
    t = t[~t['Mass'].mask]

    ### ONLY KEEP MSTAR >= 0.1
    t = t[t['Mass'] >= 0.1]

    ### FLAG (NON-)DETECTIONS
    ### REPLACE NON-DETECTION FLUXES WITH 3-SIGMA UPPER LIMITS
    t['det'] = np.repeat(0, len(t))
    t['det'][t['FCont'] / t['e_FCont'] >= 3.0] = 1
    t['MDust'][np.where(t['det'] == 0)] = 3.0 * t['e_MDust'][np.where(t['det'] == 0)]

    ### CLEAN UP
    t['Mass'].name = 'Mstar'
    t['SpType'].name = 'SpT'

    return t['Name', 'SpT', 'Mstar', 'FCont', 'MDust', 'det', 'Dis']

In [12]:
def get_tau(f1, f2, f3):

    """
    PURPOSE:    Create table for Taurus with info needed to run the 
                two-sample tests and the KME for comparing regions

                Calculates dust masses in same way as in Ansdell+2016
                Replaces non-detections with 3-sigma upper limits

    INPUT:      f1 = path to table 2 from Andrews+2013 (2013ApJ...771..129A)
                f2 = path to table 3 from Andrews+2013 (2013ApJ...771..129A)
                f2 = path to table 4 from Andrews+2013 (2013ApJ...771..129A)

    OUTPUT:     t = table that contains:
                  Source names (str)
                  Detection flags (int; 1=detection, 0=non-detection)
                  Dust masses with non-detections set to 3-sigma upper limits
                  Continuum fluxes with non-detections set to 3-sigma upper limits
                  Stellar masses and spectral types
                  Flag for observations at 890 or 1300 microns

    """

    ### GET STELLAR MASSES
    name1, mstar = [], []
    with open(f2, newline='') as f:

        reader = csv.reader(f, delimiter='\t')
        for i, row in enumerate(reader):
            if (i >= 6) & (i <= 217):

                ### REMOVE FOOTNOTES FROM NAMES
                row[0] = row[0].split('^')[0]

                ### THESE SOURCES HAD DIFFERENT NAMES IN OTHER TABLE
                if row[0] == 'JH 112 Aa':
                    row[0] = 'JH 112 A'
                if row[0] == 'JH 112 Ab':
                    row[0] = 'JH 112 B'

                ### THESE SOURCES ARE NOT IN OTHER TABLES
                if row[0] in ['J04361030+2159364', 'J04263055+2443558', 'J04335245+2612548', 'J04290068+2755033']:
                    continue

                ### THIS SOURCE WAS SEPARATED IN A & B COMPONENTS IN OTHER TABLE
                if row[0] == 'CIDA 11 AB':
                    row[0] = ['CIDA 11 A', 'CIDA 11 B']
                    row[9] = [row[9], row[9]]
                else:
                    ### SO ALL OTHERS ARE ALSO LISTS; NEEDED FOR EXTEND FUNCTION BELOW
                    row[0] = row[0].split(';')
                    row[9] = row[9].split(';')

                ### ADD TO LIST
                name1.extend(row[0])
                mstar.extend(row[9])

    ### GET FLUXES AND SPECTRAL TYPES
    name2, spt, f890, f1300, det890, det1300, note = [], [], [], [], [], [], []
    with open(f1, newline='') as f:
        
        reader = csv.reader(f, delimiter='\t')
        for i, row in enumerate(reader):
            if (i >= 6) & (i <= 215):

                ### THESE SOURCES ARE NOT IN OTHER TABLE
                if row[0] in ['J04290068+2755033']:
                    continue

                ### GET SPECTRAL TYPE
                if '(' in row[1]:
                    row[1] = row[1].split('-')[0][1:]
                spt.append(row[1].split(', ')[0].split(' +or- ')[0])

                ### GET NAME
                name2.append(row[0].split('^')[0])

                ### GET FLUXES AND FLAG DETECTIONS
                f890.append(row[3].split(' +or- ')[0].split('<')[-1])
                if '<' in row[3]:
                    det890.append(0)
                else:
                    det890.append(1)
                f1300.append(row[4].split(' +or- ')[0].split('<')[-1])
                if '<' in row[4]:
                    det1300.append(0)
                else:
                    det1300.append(1)

                ### GET NOTE ON OBSERVED OR EXTRAPOLATED
                note.append(row[5])                

    ### CREATE TABLE
    t = Table()
    t['Name'] = name1
    t['F890'] = np.array(f890).astype(float) * 1e3
    t['F1300'] = np.array(f1300).astype(float) * 1e3
    t['det890'] = det890
    t['det1300'] = det1300
    t['Mstar'] = 10**np.array(mstar).astype(float)
    t['SpT'] = spt

    ### FIGURE OUT WHICH WERE MEASURED AT WHAT WAVELENGTH
    otype = np.empty(len(t), dtype='U10')
    for i, val in enumerate(note):
        test = val.split(", ")
        if (test[0] == 'm') or (test[0] == '(m'): 
            otype[i] = 890
        else: 
            otype[i] = 1300
    t['ObsType'] = otype

    ### ONLY KEEP MSTAR >= 0.1
    t = t[t['Mstar'] >= 0.1]

    # ### REMOVE BINARIES
    # ### SINCE LUPUS AND UPPER SCO SAMPLES DIDN'T INCLUDE SECONDARY SOURCES
    # ind_bin = []
    # for i, val in enumerate(t['Name']):
    #     if i == 0:
    #         continue
    #     if (' B' in t['Name'][i]) & (' A' in t['Name'][i-1]):
    #         ind_bin.append(i)
    #     if val == 'UZ Tau Wb':
    #         ind_bin.append(i)
    # t.remove_rows(ind_bin)

    ### CALCULATE DUST MASSES USING SAME METHOD AS Ansdell+2016
    mdust, det, fcont = [], [], []
    for i, val in enumerate(t['ObsType']):
        if val == '890':
            mdust.append(calc_dust_masses.get_dustmass(1.0, 890., t['F890'][i], 140., 20.))
            det.append(t['det890'][i])
            fcont.append(t['F890'][i])
        elif val == '1300':
            mdust.append(calc_dust_masses.get_dustmass(1.0, 1300., t['F1300'][i], 140., 20.))
            det.append(t['det1300'][i])
            fcont.append(t['F1300'][i])
    t['MDust'] = mdust
    t['det'] = det
    t['FCont'] = fcont

    return t['Name', 'SpT', 'Mstar', 'FCont', 'MDust', 'det', 'ObsType']

============================= Code ==================================

In [13]:
#### LOAD IN TABLES 
TL = get_lup('../input/apjaa2846t1_mrf.txt', '../input/apjaa2846t2_mrf.txt')
TU = get_usc('../input/apjaa2b81t1_mrt.txt', '../input/apjaa2b81t4_mrt.txt')
TT = get_tau('../input/apj476413t2_ascii.txt', '../input/apj476413t3_ascii.txt', '../input/apj476413t4_ascii.txt')

In [14]:
### DISPLAY TABLES AS PANDAS DATAFRAMES
TL_df = TL.to_pandas()
TU_df = TU.to_pandas()
TT_df = TT.to_pandas()

In [15]:
# Display DataFrames
print("Lupus DataFrame:")
display(TL_df)
print("\nUpper Sco DataFrame:")
display(TU_df)
print("\nTaurus DataFrame:")
display(TT_df)

Lupus DataFrame:


,Name,SpT,Mstar,FCont,MDust,det,Dis
0,J15445789-3423392,M5.0,0.12,-0.05,0.1269,0,150
1,J15450887-3417333,M5.5,0.14,46.27,10.8740,1,150
2,J15592523-4235066,M5.0,0.12,-0.03,0.1341,0,150
3,J16000060-4221567,M4.5,0.19,2.40,0.5640,1,150
4,J16000236-4222145,M4.0,0.24,119.85,28.1662,1,150
...,...,...,...,...,...,...,...
64,Sz 97,M4.0,0.25,4.64,1.9386,1,200
65,Sz 98,K7.0,0.74,237.29,99.1394,1,200
66,Sz 99,M4.0,0.22,-0.04,0.2256,0,200
67,V1192 Sco,M4.5,0.16,0.91,0.3802,1,200



Upper Sco DataFrame:


,Name,SpT,Mstar,FCont,MDust,det
0,2MASS J15354856-2958551,M4,0.263027,1.92,0.40986,1
1,2MASS J15514032-2146103,M4,0.199526,0.76,0.16223,1
2,2MASS J15521088-2125372,M4,0.177828,0.45,0.09606,0
3,2MASS J15530132-2114135,M4,0.208930,5.78,1.23384,1
4,2MASS J15534211-2049282,M3.5,0.269153,2.93,0.62546,1
...,...,...,...,...,...,...
70,2MASS J16163345-2521505,M0.5,0.512861,2.88,0.61478,1
71,2MASS J16181618-2619080,M4.5,0.177828,0.36,0.07685,0
72,2MASS J16181904-2028479,M4.75,0.162181,4.62,0.98622,1
73,2MASS J16270942-2148457,M4.5,0.162181,2.87,0.61265,1



Taurus DataFrame:


,Name,SpT,Mstar,FCont,MDust,det,ObsType
0,IRAS 04108+2910,M0,0.588844,19.8,10.96599,0,1300
1,FM Tau,M0,0.602560,28.8,5.89798,1,890
2,FN Tau,M5,0.234423,36.5,7.47487,1,890
3,CW Tau,K3,1.584893,160.1,32.78703,1,890
4,CIDA 1,M5.5,0.128825,13.5,7.47681,1,1300
...,...,...,...,...,...,...,...
176,CIDA 11 A,M3.5,0.416869,8.0,1.63833,0,890
177,CIDA 11 B,M4.5,0.416869,8.0,1.63833,0,890
178,RW Aur A,K2.5,1.513561,56.0,11.46829,1,890
179,RW Aur B,K5,1.047129,13.2,2.70324,1,890


In [16]:
### WRITE FILES
TL.write('../output/data_lup.txt', format='ascii.ipac', overwrite=True)
TU.write('../output/data_usc.txt', format='ascii.ipac', overwrite=True)
TT.write('../output/data_tau.txt', format='ascii.ipac', overwrite=True)